# Data Preprocessing 

To prepare our data for analysis, we need to perform data preprocessing/data wrangling. In this section, we will learn how to clean and reformat data (e.g., renaming columns and fixing data type mismatches), restructure/reshape it, and enrich it (e.g., discretizing columns, calculating aggregations, and combining data sources).

### Yellow Taxi Trip Data
In this section, we will take a look at creating, renaming, and dropping columns; type conversion; and sorting &ndash; all of which make our analysis easier. We will be working with the 2019 Yellow Taxi Trip Data provided by NYC Open Data.

In [ ]:
import pandas as pd

taxis = pd.read_csv('data/2019_Yellow_Taxi_Trip_Data.csv')
taxis.head()

In [ ]:
taxis.isnull().values.any()

*Source: [NYC Open Data](https://data.cityofnewyork.us/Transportation/2019-Yellow-Taxi-Trip-Data/2upf-qytp)*

### Dropping columns
Let's start by dropping the ID columns and the `store_and_fwd_flag` column, which we won't be using.

In [ ]:
mask = taxis.columns.str.contains('id$|store_and_fwd_flag', regex=True)
columns_to_drop = taxis.columns[mask]
columns_to_drop

*here we use [regular expressions](https://www.regular-expressions.info/) to reference all columns that include 'id'.*

In [ ]:
taxis = taxis.drop(columns=columns_to_drop)
taxis.head()

*Tip: Another way to do this is to select the columns we want to keep: `taxis.loc[:,~mask]`.*

### Renaming columns

Next, let's rename the datetime columns:

In [ ]:
taxis.columns

In [ ]:
taxis = taxis.rename(
    columns={
        'tpep_pickup_datetime': 'pickup', 
        'tpep_dropoff_datetime': 'dropoff'
    }
)
taxis.columns

### Type conversion

Notice anything off with the data types?

In [ ]:
taxis.dtypes

In [ ]:
taxis.head()

Both `pickup` and `dropoff` should be stored as datetimes. Let's fix this:

In [ ]:
taxis[['pickup', 'dropoff']] = taxis[['pickup', 'dropoff']].apply(pd.to_datetime)
taxis.dtypes

*Tip: There are other ways to perform type conversion. For numeric values, we can use the `pd.to_numeric()` function, and we will see the `astype()` method, which is a more generic method, a little later.*

### Creating new features (columns)

Let's calculate the following for each row:

1. elapsed time of the trip
2. the tip percentage
3. the total taxes, tolls, fees, and surcharges
4. the average speed of the taxi

In [ ]:
taxis = taxis.assign(
    elapsed_time=lambda x: x.dropoff - x.pickup, # 1
    cost_before_tip=lambda x: x.total_amount - x.tip_amount,
    tip_pct=lambda x: x.tip_amount / x.cost_before_tip, # 2
    fees=lambda x: x.cost_before_tip - x.fare_amount, # 3
    avg_speed=lambda x: x.trip_distance.div( x.elapsed_time.dt.total_seconds() / 60 / 60 ) # 4
)

In [ ]:
#Alternatively you can do it one variable by one
taxis['elapsed_time_alt'] = taxis.dropoff - taxis.pickup
taxis['cost_before_tip_alt'] = taxis.total_amount - taxis.tip_amount
taxis['tip_pct_alt'] = taxis.tip_amount / taxis.cost_before_tip
taxis['fees_alt'] = taxis.cost_before_tip - taxis.fare_amount
taxis['avg_speed_alt'] = taxis.trip_distance.div(taxis.elapsed_time.dt.total_seconds() / 60 / 60 )

In [ ]:
taxis.head()

In [ ]:
mask = taxis.columns.str.contains('_alt$', regex=True)
columns_to_drop = taxis.columns[mask]
taxis = taxis.drop(columns=columns_to_drop)

*Tip: New to `lambda` functions? These small, anonymous functions can receive multiple arguments, but can only contain one expression (the return value). You will see these a lot in pandas code. Read more about them [here](https://realpython.com/python-lambda/).*

Our new columns get added to the right:

In [ ]:
taxis.head(2)

Some things to note:
- We used `lambda` functions to 
    1) avoid typing `taxis` repeatedly and 
    2) be able to access the `cost_before_tip` and `elapsed_time` columns in the same method that we create them.


- To create a single new column, we can also use `df['new_col'] = <values>`.

### Sorting by values

We can use the `sort_values()` method to sort based on any number of columns:

In [ ]:
taxis.sort_values(['passenger_count', 'pickup'], ascending=[False, True]).head()

To pick out the largest/smallest rows, use `nlargest()` / `nsmallest()` instead. Looking at the 3 trips with the longest elapsed time, we see some possible data integrity issues:

In [ ]:
taxis.nlargest(3, 'elapsed_time')

### [Exercise 1.1](../exercises/data_preprocessing_exercise.ipynb##Exercise-1.1)

##### Read in the meteorite data from the `Meteorite_Landings.csv` file, rename the `mass (g)` column to `mass`, and drop all the latitude and longitude columns. Sort the result by mass in descending order.

## Working with the index

So far, we haven't really worked with the index because it's just been a row number; however, we can change the values we have in the index to access additional features of the pandas library.

### Setting and sorting the index

Currently, we have a RangeIndex, but we can switch to a DatetimeIndex by specifying a datetime column when calling `set_index()`:

In [ ]:
taxis = taxis.set_index('pickup')
taxis.head(3)

Since we have a sample of the full dataset, let's sort the index to order by pickup time:

In [ ]:
taxis = taxis.sort_index()

*Tip: `taxis.sort_index(axis=1)` will sort the columns by name. The `axis` parameter is present throughout the pandas library: `axis=0` targets rows and `axis=1` targets columns.*

We can now select ranges from our data based on the datetime the same way we did with row numbers:

In [ ]:
taxis['2019-10-23 07:45':'2019-10-23 08']

When not specifying a range, we use `loc[]`:

In [ ]:
taxis.loc['2019-10-23']

### Resetting the index

We will be working with time series later this section, but sometimes we want to reset our index to row numbers and restore the columns. We can make `pickup` a column again with the `reset_index()` method:

In [ ]:
taxis = taxis.reset_index()
taxis.head()

### [Exercise 1.2](../exercises/data_preprocessing_exercise.ipynb#Exercise-1.2)

##### Using the meteorite data from the `Meteorite_Landings.csv` file, update the `year` column to only contain the year, convert it to a numeric data type, and create a new column indicating whether the meteorite was observed falling before 1970. Set the index to the `id` column and extract all the rows with IDs between 10,036 and 10,040 (inclusive) with `loc[]`.

###### **Hint 1**: Use `year.str.slice()` to grab a substring.

###### **Hint 2**: Make sure to sort the index before using `loc[]` to select the range.

###### **Bonus**: There's a data entry error in the `year` column. Can you find it? (Don't spend too much time on this.)

## Reshaping data

The taxi dataset we have be working with is in a format conducive to an analysis. This isn't always the case. Let's now take a look at the TSA traveler throughput data, which compares 2021 throughput to the same day in 2020 and 2019:

In [ ]:
tsa = pd.read_csv('data/tsa_passenger_throughput.csv', parse_dates=['Date'])
tsa.head()

*Source: [TSA.gov](https://www.tsa.gov/coronavirus/passenger-throughput)*

First, we will lowercase the column names and take the first word (e.g., `2021` for `2021 Traveler Throughput`) to make this easier to work with:

In [ ]:
tsa = tsa.rename(columns=lambda x: x.lower().split()[0])
tsa.head()

Now, we can work on reshaping it.

### Melting

Melting helps convert our data into long format. Now, we have all the traveler throughput numbers in a single column:

In [ ]:
tsa_melted = tsa.melt(
    id_vars='date', # column that uniquely identifies a row (can be multiple)
    var_name='year', # name for the new column created by melting
    value_name='travelers' # name for new column containing values from melted columns
)
tsa_melted.sample(5, random_state=1) # show some random entries

To convert this into a time series of traveler throughput, we need to replace the year in the `date` column with the one in the `year` column. Otherwise, we are marking prior years' numbers with the wrong year.

In [ ]:
tsa_melted = tsa_melted.assign(
    date=lambda x: pd.to_datetime(x.year + x.date.dt.strftime('-%m-%d'))
)
tsa_melted.sample(5, random_state=1)

This leaves us with some null values (the dates that aren't present in the dataset):

In [ ]:
tsa_melted.sort_values('date').tail(3)

These can be dropped with the `dropna()` method:

In [ ]:
tsa_melted = tsa_melted.dropna()
tsa_melted.sort_values('date').tail(3)

### Pivoting

Using the melted data, we can pivot the data to compare TSA traveler throughput on specific days across years:

In [ ]:
#tsa_melted.info()

In [ ]:
tsa_pivoted = tsa_melted\
    .query('date.dt.month == 3 and date.dt.day <= 10')\
    .assign(day_in_march=lambda x: x.date.dt.day)\
    .pivot(index='year', columns='day_in_march', values='travelers')
tsa_pivoted

**Important**: We aren't covering the `unstack()` and `stack()` methods, which are additional ways to pivot and melt, respectively. These come in handy when we have a multi-level index (e.g., if we ran `set_index()` with more than one column). More information can be found [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html).

### Transposing

The `T` attribute provides a quick way to flip rows and columns.

In [ ]:
tsa_pivoted.T

### Merging

We typically observe changes in air travel around the holidays, so adding information about the dates in the TSA dataset provides more context. The `holidays.csv` file contains a few major holidays in the United States:

In [ ]:
holidays = pd.read_csv('data/holidays.csv', parse_dates=True, index_col='date')
holidays.loc['2019']

Merging the holidays with the TSA traveler throughput data will provide more context for our analysis:

In [ ]:
tsa_melted_holidays = tsa_melted\
    .merge(holidays, left_on='date', right_index=True, how='left')\
    .sort_values('date')
tsa_melted_holidays.head()

In [ ]:
tsa_melted_holidays.query(
    'year == "2019" and '
    '(holiday == "Thanksgiving" or holiday.str.contains("Christmas"))'
)

*Tip: There are many parameters for this method, so be sure to check out the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html). To append rows, take a look at the `pd.concat()` function.*

We can take this a step further by marking a few days before and after each holiday as part of the holiday. This would make it easier to compare holiday travel across years and look for any uptick in travel around the holidays:

In [ ]:
tsa_melted_holiday_travel = tsa_melted_holidays.assign(
    holiday=lambda x:
        x.holiday\
            .fillna(method='ffill', limit=1)\
            .fillna(method='bfill', limit=2)
)

*Tip: Check out the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) for the full list of functionality available with the `fillna()` method.*

Notice that we now have values for the day after each holiday and the two days prior. Thanksgiving in 2019 was on November 28th, so the 26th, 27th, and 29th were filled. Since we are only replacing null values, we don't override Christmas Day with the forward fill of Christmas Eve:

In [ ]:
tsa_melted_holiday_travel.query(
    'year == "2019" and '
    '(holiday == "Thanksgiving" or holiday.str.contains("Christmas"))'
)

## Aggregations and grouping

After reshaping and cleaning our data, we can perform aggregations to summarize it in a variety of ways. In this section, we will explore using pivot tables, crosstabs, and group by operations to aggregate the data.

### Pivot tables
We can build a pivot table to compare holiday travel across the years in our dataset:

In [ ]:
tsa_melted_holiday_travel.pivot_table(
    index='year', columns='holiday', 
    values='travelers', aggfunc='sum'
)

We can use the `pct_change()` method on this result to see which holiday travel periods saw the biggest change in travel:

In [ ]:
tsa_melted_holiday_travel.pivot_table(
    index='year', columns='holiday', 
    values='travelers', aggfunc='sum'
).pct_change()

Let's make one last pivot table with column and row subtotals, along with some formatting improvements. First, we set a display option for all floats:

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)

Next, we group together Christmas Eve and Christmas Day, likewise for New Year's Eve and New Year's Day, and create the pivot table:

In [ ]:
import numpy as np

tsa_melted_holiday_travel.assign(
    holiday=lambda x: np.where(
        x.holiday.str.contains('Christmas|New Year', regex=True), 
        x.holiday.str.replace('Day|Eve', '', regex=True).str.strip(), 
        x.holiday
    )
).pivot_table(
    index='year', columns='holiday', 
    values='travelers', aggfunc='sum', 
    margins=True, margins_name='Total'
)

Before moving on, let's reset the display option:

In [ ]:
pd.reset_option('display.float_format')

*Tip: Read more about options in the documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).*

### [Exercise 1.3](../exercises/data_preprocessing.ipynb#Exercise-1.3)

##### Using the meteorite data from the `Meteorite_Landings.csv` file, create a pivot table that shows both the number of meteorites and the 95th percentile of meteorite mass for those that were found versus observed falling per year from 2005 through 2009 (inclusive). Hint: Be sure to convert the `year` column to a number as we did in the previous exercise.

### Crosstabs
The `pd.crosstab()` function provides an easy way to create a frequency table. Here, we count the number of low-, medium-, and high-volume travel days per year, using the `pd.cut()` function to create three travel volume bins of equal width:

In [ ]:
pd.crosstab(
    index=pd.cut(
        tsa_melted_holiday_travel.travelers, 
        bins=3, labels=['low', 'medium', 'high']
    ),
    columns=tsa_melted_holiday_travel.year,
    rownames=['travel_volume']
)

*Tip: The `pd.cut()` function can also be used to specify custom bin ranges. For equal-sized bins based on quantiles, use the `pd.qcut()` function instead.*

Note that the `pd.crosstab()` function supports other aggregations provided you pass in the data to aggregate as `values` and specify the aggregation with `aggfunc`. You can also add subtotals and normalize the data. See the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) for more information.

### Group by operations
Rather than perform aggregations, like `mean()` or `describe()`, on the full dataset at once, we can perform these calculations per group by first calling `groupby()`:

In [ ]:
tsa_melted_holiday_travel.groupby('year').describe()

Groups can also be used to perform separate calculations per subset of the data. For example, we can find the highest-volume travel day per year using `rank()`:

In [ ]:
tsa_melted_holiday_travel.assign(
    travel_volume_rank=lambda x: x.groupby('year').travelers.rank(ascending=False)
).sort_values(['travel_volume_rank', 'year']).head(3)

The previous two examples called a single method on the grouped data, but using the `agg()` method we can specify any number of them:

In [ ]:
tsa_melted_holiday_travel.assign(
    holiday_travelers=lambda x: np.where(~x.holiday.isna(), x.travelers, np.nan),
    non_holiday_travelers=lambda x: np.where(x.holiday.isna(), x.travelers, np.nan),
    year=lambda x: pd.to_numeric(x.year)
).select_dtypes(include='number').groupby('year').agg(['mean', 'std'])

*Tip: The `select_dtypes()` method makes it possible to select columns by their data type. We can specify the data types to `exclude` and/or `include`.*

In addition, we can specify which aggregations to perform on each column:

In [ ]:
tsa_melted_holiday_travel.assign(
    holiday_travelers=lambda x: np.where(~x.holiday.isna(), x.travelers, np.nan),
    non_holiday_travelers=lambda x: np.where(x.holiday.isna(), x.travelers, np.nan)
).groupby('year').agg({
    'holiday_travelers': ['mean', 'std'], 
    'holiday': ['nunique', 'count']
})

We are only scratching the surface; some additional functionalities to be aware of include the following:
- We can group by multiple columns &ndash; this creates a hierarchical index.
- Groups can be excluded from calculations with the `filter()` method.
- We can group on content in the index using the `level` or `name` parameters e.g., `groupby(level=0)` or `groupby(name='year')`.
- We can group by date ranges if we use a `pd.Grouper()` object.

Be sure to check out the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) for more details.

<!-- ### [Exercise 1.4](../exercises/data_preprocessing.ipynb#Exercise-1.4)

##### Using the meteorite data from the `Meteorite_Landings.csv` file, compare summary statistics of the mass column for the meteorites that were found versus observed falling. -->

## Time series

When working with time series data, pandas provides us with additional functionality to not just compare the observations in our dataset, but to use their relationship in time to analyze the data. In this section, we will see a few such operations for selecting date/time ranges, calculating changes over time, performing window calculations, and resampling the data to different date/time intervals.

### Selecting based on date and time

Let's switch back to the `taxis` dataset, which has timestamps of pickups and dropoffs. First, we will set the `dropoff` column as the index and sort the data:

In [ ]:
taxis = taxis.set_index('dropoff').sort_index()

We saw earlier that we can slice on the datetimes:

In [ ]:
taxis['2019-10-24 12':'2019-10-24 13']

We can also represent this range with shorthand. Note that we must use `loc[]` here:

In [ ]:
taxis.loc['2019-10-24 12']

However, if we want to look at this time range across days, we need another strategy.

We can pull out the dropoffs that happened between a certain time range on *any* day with the `between_time()` method:

In [ ]:
taxis.between_time('12:00', '13:00')

*Tip: The `at_time()` method can be used to extract all entries at a given time (e.g., 12:35:27).*

Finally, `head()` and `tail()` limit us to a number of rows, but we may be interested in rows within the first/last 2 hours (or any other time interval) of the data, in which case, we should use `first()` / `last()`:

In [ ]:
taxis.first('2H')

*Tip: Available date/time offsets can be found in the pandas documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).* 

For the rest of this section, we will be working with the TSA traveler throughput data. Let's start by setting the index to the `date` column:

In [ ]:
tsa_melted_holiday_travel = tsa_melted_holiday_travel.set_index('date')

### Calculating change over time

In [ ]:
help(pd.DataFrame.diff)

In [ ]:
tsa_melted_holiday_travel.loc['2020'].assign(
    one_day_change=lambda x: x.travelers.diff(),
    seven_day_change=lambda x: x.travelers.diff(7),
).head(10)

*Tip: To perform operations other than subtraction, take a look at the `shift()` method. It also makes it possible to perform operations across columns.*

### Resampling
We can use resampling to aggregate time series data to a new frequency:

In [ ]:
tsa_melted_holiday_travel['2019':'2021-Q1'].select_dtypes(include='number')\
    .resample('Q').agg(['sum', 'mean', 'std'])

### Window calculations

Window calculations are similar to group by calculations except the group over which the calculation is performed isn't static &ndash; it can move or expand. Pandas provides functionality for constructing a variety of windows, including moving/rolling windows, expanding windows (e.g., cumulative sum or mean up to the current date in a time series), and exponentially weighted moving windows (to weight closer observations more than further ones). We will only look at rolling and expanding calculations here.

In [ ]:
#help(pd.Series.rolling)

Performing a window calculation is very similar to a group by calculation &ndash; we first define the window, and then we specify the aggregation:

In [ ]:
tsa_melted_holiday_travel.loc['2020'].assign(
    **{
        '7D MA': lambda x: x.rolling('7D').travelers.mean(),
        'YTD mean': lambda x: x.expanding().travelers.mean()
      }
).head(10)

To understand what's happening, it's best to visualize the original data and the result, so here's a sneak peek of plotting with pandas. First, some setup to embed SVG plots in the notebook:

In [ ]:
import matplotlib_inline
from utils import mpl_svg_config

matplotlib_inline.backend_inline.set_matplotlib_formats(
    'svg', # output images using SVG format
    **mpl_svg_config('section-2') # optional: configure metadata
)

*Tip: For most use cases, only the first argument is necessary &ndash; we will discuss the second argument in more detail in the next section.*

Now, we call the `plot()` method to visualize the data:

In [ ]:
_ = tsa_melted_holiday_travel.loc['2020'].assign(
    **{
        '7D MA': lambda x: x.rolling('7D').travelers.mean(),
        'YTD mean': lambda x: x.expanding().travelers.mean()
      }
).plot(title='2020 TSA Traveler Throughput', ylabel='travelers', alpha=0.8)

Other types of windows:
- [exponentially weighted moving](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html): use the `ewm()` method
- [custom](https://pandas.pydata.org/docs/user_guide/window.html#window-custom-rolling-window): create a subclass of `pandas.api.indexers.BaseIndexer` or use a pre-built one in `pandas.api.indexers`

<!-- ### [Exercise 1.5](../exercises/data_preprocessing.ipynb#Exercise-1.5)

##### Using the taxi trip data in the `2019_Yellow_Taxi_Trip_Data.csv` file, resample the data to an hourly frequency based on the dropoff time. Calculate the total `trip_distance`, `fare_amount`, `tolls_amount`, and `tip_amount`, then find the 5 hours with the most tips. -->

# The Feature Engineering Checklist :

There are a number of preprocessing steps a data scientist should consider when creating their Analytical Base Table (ABT). The following list covers most of these potential steps, but you should allways consider the purpose of your study.


- **Feature Scaling**: This is a technique used to scale numerical features to ensure they are on the same scale. There are several techniques for feature scaling such as Min-Max scaling, Z-score normalization, and log transformation.

- **Feature Aggregation**: This involves creating new features by aggregating or summarizing existing ones. For example, if we have a dataset of sales data with daily sales figures, we can create a new feature for weekly sales by summing up the daily sales figures for each week.

- **Feature Decomposition**: This involves breaking down complex features into simpler ones. For example, if we have a feature for a date, we can decompose it into day, month, and year features.

- **One-Hot Encoding**: This is a technique used for categorical variables. It converts each category into a binary column. For example, if we have a categorical feature "color" with values "red", "green" and "blue", one-hot encoding will create three columns - "color_red", "color_green" and "color_blue" - and assign a binary value (0 or 1) to each column based on the presence or absence of the category.

- **Domain-Specific Feature Engineering**: This involves using domain-specific knowledge to engineer features. For example, in a medical diagnosis problem, domain-specific features may include blood pressure, pulse rate, and cholesterol levels.

- **Feature Interactions**: This involves creating new features by combining existing features. For example, if we have two features - age and income - we can create a new feature by multiplying them together.

- **Feature Binning**: This involves grouping continuous features into discrete bins or categories. This can help capture non-linear relationships between the features and the target variable.

- **Feature Extraction**: This involves creating new features from existing ones using techniques such as Principal Component Analysis (PCA), Singular Value Decomposition (SVD), or Non-negative Matrix Factorization (NMF).


- **Feature Selection**: This involves selecting the most relevant features for the problem at hand. This can be done using various techniques such as correlation analysis, mutual information, and feature importance measures.


- **Text Feature Engineering**: For text data, feature engineering may involve techniques such as word tokenization, stop-word removal, stemming or lemmatization, and vectorization techniques such as Bag-of-Words, TF-IDF, or word embeddings.

- **Time Series Feature Engineering**: For time series data, feature engineering may involve techniques such as lag features, rolling window statistics, seasonal decomposition, or Fourier transforms.

- **Image Feature Engineering**: For image data, feature engineering may involve techniques such as color histograms, edge detection, texture analysis, or deep learning-based feature extraction using convolutional neural networks (CNNs).


- **Handling Missing Data**: This involves imputing missing values in the data using techniques such as mean imputation, mode imputation, or regression imputation. *(this can also be seen as part of the cleaning process)*

- **Outlier Detection**: This involves identifying and handling outliers in the data using techniques such as Z-score or interquartile range (IQR) method. *(this can also be seen as part of the cleaning process)*



return to [overview](../00_overview.ipynb)